In [1]:
import pandas as pd
import numpy as np
import pickle

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1500

df = pd.read_excel('C:/Users/Anastasiya.Mittseva/PycharmProjects/ProjectPredictSolidity/prepared_data.xlsx')

df.head()

,Сталь,С,Mn,Si,S,P,Cr,Ni,Cu,Mo,V,Nb,Ti,Al,N,B,Сэкв,Тнагрева,"время выдержки, мин",параметр закалки,Ас1,Ас3,V800-500 °C/c,%Ф,%П,%Б,%М,%А остаточный,HV,HRC
0,32г2,0.32,1.1,0.27,0.02,0.017,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549333,850,10,810.626512,735.0,795.0,0.11,NaN,NaN,NaN,NaN,NaN,180.0,NaN
1,32г2,0.32,1.1,0.27,0.02,0.017,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549333,850,10,810.626512,735.0,795.0,0.31,NaN,NaN,NaN,NaN,NaN,180.0,NaN
2,32г2,0.32,1.1,0.27,0.02,0.017,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549333,850,10,810.626512,735.0,795.0,1.00,NaN,NaN,NaN,NaN,NaN,180.0,NaN
3,32г2,0.32,1.1,0.27,0.02,0.017,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549333,850,10,810.626512,735.0,795.0,3.40,NaN,NaN,NaN,NaN,NaN,195.0,NaN
4,32г2,0.32,1.1,0.27,0.02,0.017,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549333,850,10,810.626512,735.0,795.0,7.00,NaN,NaN,NaN,NaN,NaN,200.0,NaN


## Подготовка данных

In [2]:
def convertHRCtoHV(x):
    y = 0.0001076546813696400 * x ** 4 - 0.01404920211793480 * x ** 3 + 0.7989548243359580 * x ** 2 - 13.82475417563670 * x + 289.8692821768870
    return y

def repl(x):
    if np.isnan(x):
        return round(convertHRCtoHV(df['HRC'].loc[x.index]),2)
    else:
        return x

In [3]:
df['HRC'] = df['HRC'].map(convertHRCtoHV)

df['HV'] = df['HV'].combine_first(df['HRC'])

df = df.drop('HRC',1)

df = df.fillna(0)

df = df[~(df['HV'] == 0.0)]

df = df.drop(df[df['%А остаточный'] != 0].index)

df = df.drop('%А остаточный',1)

df = df[~(df['V800-500 °C/c']== 0.0)]

df = df.drop(df[(df['%Ф']+df['%П']+df['%Б']+df['%М']) != 100].index)

df = df.reset_index()

df = df.drop('index',1)

df.head()

,Сталь,С,Mn,Si,S,P,Cr,Ni,Cu,Mo,V,Nb,Ti,Al,N,B,Сэкв,Тнагрева,"время выдержки, мин",параметр закалки,Ас1,Ас3,V800-500 °C/c,%Ф,%П,%Б,%М,HV
0,X65NbV,0.11,1.02,0.28,0.0015,0.009,0.13,0.01,0.18,0.09,0.071,0.018,0.0,0.026,0.0,0.0,0.354467,920,10,875.66189,0.0,0.0,1.0,84.0,16.0,0.0,0.0,161.0
1,X65NbV,0.11,1.02,0.28,0.0015,0.009,0.13,0.01,0.18,0.09,0.071,0.018,0.0,0.026,0.0,0.0,0.354467,920,10,875.66189,0.0,0.0,5.0,82.0,12.0,0.0,6.0,175.0
2,X65NbV,0.11,1.02,0.28,0.0015,0.009,0.13,0.01,0.18,0.09,0.071,0.018,0.0,0.026,0.0,0.0,0.354467,920,10,875.66189,0.0,0.0,10.0,79.0,8.0,5.0,8.0,188.0
3,X65NbV,0.11,1.02,0.28,0.0015,0.009,0.13,0.01,0.18,0.09,0.071,0.018,0.0,0.026,0.0,0.0,0.354467,920,10,875.66189,0.0,0.0,20.0,69.0,0.0,21.0,10.0,200.0
4,X65NbV,0.11,1.02,0.28,0.0015,0.009,0.13,0.01,0.18,0.09,0.071,0.018,0.0,0.026,0.0,0.0,0.354467,920,10,875.66189,0.0,0.0,40.0,51.0,0.0,36.0,13.0,225.0


## GRNN 

In [4]:
from sklearn.model_selection import  train_test_split, GridSearchCV
from sklearn import model_selection, preprocessing
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, matthews_corrcoef, median_absolute_error
from neupy import algorithms, estimators, environment
from scipy.stats import pearsonr

ls_train_col = ['С', 'Mn', 'Si', 'S', 'P', 'Cr', 'Ni', 'Cu', 'Mo', 'V', 'Nb', 'Ti', 'Al', 'N', 'B', 'Сэкв','параметр закалки','V800-500 °C/c']
targets = ['%Ф','%П','%Б', '%М']

x_input = df[ls_train_col]
y_input = df[targets]

x, x_valid, y, y_valid = train_test_split(preprocessing.minmax_scale(x_input, feature_range = (0 , 1)),preprocessing.minmax_scale(y_input, feature_range = (0 , 1)),test_size=0.3)

def print_and_return_score(x, y, est):
    y_pred = est.predict(x)
#     mae = mean_absolute_error(y, y_pred)
#     mse = mean_squared_error(y, y_pred)
#     r2 = r2_score(y, y_pred)
    corr = pearsonr(y, y_pred)   
    
#     print('R^2: ', round(r2, 2))
#     print('MAE: ', round(mae, 2))
#     print('MSE: ', round(mse, 2))
    print('Corr: ', round(corr[0][0], 2))
    
    return corr[0][0]

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [5]:
x_train, x_test, y_train, y_test = train_test_split(preprocessing.minmax_scale(x, feature_range = (0 , 1)),preprocessing.minmax_scale(y, feature_range = (0 , 1)),test_size=0.3)

In [6]:
nw = algorithms.GRNN(std=0.05, verbose=False)

In [7]:
y_train

array([[0.02061856, 0.        , 0.4       , 0.58      ],
       [0.07216495, 0.93      , 0.        , 0.        ],
       [0.67010309, 0.15      , 0.1       , 0.1       ],
       ...,
       [0.        , 0.        , 0.02      , 0.98      ],
       [0.30927835, 0.        , 0.68      , 0.02      ],
       [0.        , 0.        , 0.9       , 0.1       ]])

In [62]:
nw.train(x_train, y_train)

ValueError: Target value must be one dimensional array

In [58]:
corr_test = print_and_return_score(x_test, y_test, nw)

(array([0.12788588]), array([0.03605275]))
Corr:  0.13
